In [291]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode 
from math import sqrt
from skimage.measure import find_contours
import matplotlib.pyplot as plt 
from skimage import data, color, img_as_ubyte 
from skimage.feature import canny 
from skimage.transform import hough_ellipse 
from skimage.draw import ellipse_perimeter 
from skimage.draw import rectangle

In [292]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [293]:
def opening(img,ST):
    imgEroded=cv2.erode(img,ST)
    return cv2.dilate(imgEroded,ST)
def closing(img,ST):
    imgDilated=cv2.dilate(img,ST)
    return cv2.erode(imgDilated,ST)

In [294]:
img= cv2.imread("note.png",0) 

# cv2.imwrite("test.jpg",img)
# print(img[:,1])
if(np.amax(img)==1):
    # img=img.astype("uint8")
    # cv2.imwrite("test1.jpg",img)
    print("hi")
else:
    img=(img/255)
    # cv2.imwrite("test2.png",img*255)
# print(img[:,1])
img2= np.copy(img)
img=1-img


# cv2.imwrite("test3.jpg",img*255)
# print(img)
proj = np.sum(img,1)
#proj=1-proj
maxProjection = np.max(proj) 
w = 500 
result = np.zeros(img.shape) 
result2 = np.zeros(img.shape) 
width=img.shape[1] 
binOrig=np.copy(img) 
# Draw a line for each row 
for row in range(img.shape[0]):
    result2[row,0:int(proj[row])]=1
    if(proj[row]>.8*maxProjection): 
        cv2.line(result, (0,row), (int(proj[row]*(maxProjection/width)),row), (255,255,255), 1) 
        
cv2.imwrite('result2.png', result2*255) 
cv2.imwrite('result.png', result) 
result=result2
result=1-result
print(proj.shape, img.shape)

(277,) (277, 682)


In [295]:
def calcStaffPos(rowHist):
    n,m=rowHist.shape
    thres=int(2*(maxProjection)/4)
    print(thres,m)
    start=end=0
    print(rowHist)
    width=[]
    #rowHist=(rowHist/255).astype("uint8")
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                print(start,end)
                print((start+end)/2)
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    
    return np.array(width),np.array(peaksMid)

width,peaksMids=calcStaffPos(result)
print(width,peaksMids)

173 682
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
15 17
16.0
22 24
23.0
30 32
31.0
38 40
39.0
45 48
46.5
82 84
83.0
90 92
91.0
98 100
99.0
105 108
106.5
113 115
114.0
150 152
151.0
158 160
159.0
166 168
167.0
174 176
175.0
181 183
182.0
218 220
219.0
226 228
227.0
234 236
235.0
241 243
242.0
249 251
250.0
[2 2 2 2 3 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2] [ 16  23  31  39  46  83  91  99 106 114 151 159 167 175 182 219 227 235
 242 250]


In [296]:
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum<=thresPixel):
           # print(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=0
    return imgOriginal

img2=1-img2
ST=np.ones((1,3))
img2=cv2.dilate(img2,ST)

for i in range(len(peaksMids)):
    img2=removeStaffRow(img2,peaksMids[i],width[i])

ST=np.ones((2,1))
img2=cv2.dilate(img2,ST)
img2=1-img2
cv2.imwrite("outing.png",img2*255)


True

In [297]:
def detectStaffOrient(img,Xmin,Xmax,Ymin,Ymax,index):
    countStaff=0
    #imgSymbol=img[int(Ymin):int(Ymax+1),int(Xmin):int(Xmax+1)]
    # print(imgSymbol)
    img_rgb = cv2.imread("try.png",1)  
    # It is need to be convert it to grayscale   
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)   
    template = cv2.imread("template.png",0) 
    width,height=template.shape
    match = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    position = np.where(match >= threshold)
    print(match.shape,img_rgb.shape)
    print(position)
    # x1=position[0][0]
    # y1=position[0][1]]
    # img_rgb[y1][x1]=[155,34,0]]
    for point in zip(*position[::-1]): #draw the rectangle around the matched template
        cv2.rectangle(img_rgb, point, (point[0] + width, point[1] + height), (153, 22, 0), 0)
    
    cv2.imwrite("circleDetected.png",img_rgb)
    for i in peaksMids:
        if(i<=Ymax and i >= Ymin):
            countStaff+=1

In [298]:
contours = find_contours(img2, 0.8)
index=0
for contour in contours:
    x = contour[:,1]
    y = contour[:,0]
    [Xmin, Xmax, Ymin, Ymax] = [np.amin(x), np.amax(x), np.amin(y), np.amax(y)]
    if(index==50):
       # print(index)
        rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=img.shape)
        imgSymbol=img2[int(Ymin):int(Ymax+1),int(Xmin):int(Xmax+1)]
        cv2.imwrite("try.png",imgSymbol*255)
        #img2[rr.astype(int), cc.astype(int)] = 0
        #imgSymbol=img2[int(Ymin+(0.7*distY)):int(Ymax)+1,int(Xmin):int(Xmax)+1]
        #cv2.imwrite("template.png",imgSymbol*255)
        detectStaffOrient(img2,int(Xmin),int(Xmax), int(Ymin),int(Ymax),index)
        # distY=Ymax-Ymin
        # distX=Xmax-Xmin

    index+=1


cv2.imwrite("rects.png",img2*255)


50
(24, 1) (33, 13, 3)
(array([22, 23]), array([0, 0]))


True

In [299]:

# Load picture, convert to grayscale and detect edges 

#another function


# image_rgb = data.coffee()[0:220, 160:420] 
# image_gray = color.rgb2gray(imgSymbol) 
# edges = canny(image_gray, sigma=2.0, low_threshold=0.55, high_threshold=0.8) 
# cv2.imwrite("e.png",edges*255)
# result = hough_ellipse(edges, accuracy=20, threshold=250, min_size=100, max_size=120) 
# if(len(result)):
#     print("P")
#     result.sort(order='accumulator') 
#     best = list(result[-1]) 
#     yc, xc, a, b = [int(round(x)) for x in best[1:5]] 
#     orientation = best[5]
#     cy, cx = ellipse_perimeter(yc, xc, a, b, orientation) 
#     image_rgb[cy, cx] = (0, 0, 255) 
#     edges = color.gray2rgb(img_as_ubyte(edges)) 
#     edges[cy, cx] = (250, 0, 0)
#     fig2, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(8, 4), sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'}) 
#     ax1.set_title('Original picture') 
#     ax1.imshow(image_rgb) 
#     ax2.set_title('Edge (white) and result (red)') 
#     ax2.imshow(edges) 
#     plt.show() 